In [4]:
mycursor.execute("DROP DATABASE CIRCADIEM;");

In [1]:
import mysql.connector     #python library for connecting to server
import csv
import datetime

import openpyxl
import decimal

connection  = mysql.connector.connect(user='root', password='password123',host='127.0.0.1')

In [2]:
# Create the database and use database
mycursor = connection.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS CIRCADIEM")
connection.database = 'CIRCADIEM'

# Import Functions

In [40]:
# mycursor and connection must be initialized for this function to work. (The correct database should be connected to)

def import_database(table_name, data):
    # Get the headings from the mysql table
    sql_heading_Query = "SHOW COLUMNS FROM {}".format(table_name)
    mycursor.execute(sql_heading_Query)
    heading_data = mycursor.fetchall()
    
    sql_string_heading = "INSERT IGNORE INTO {} (".format(table_name) # INSERT IGNORE is used to avoid writing over previously inserted data points (This will ensure the timestamps are correct)
    for a in range (0, len(heading_data) - 1): # (-1) For the allowance of adding commas to the string
        sql_string_heading = sql_string_heading + heading_data[a][0] + ","
    sql_string_heading = sql_string_heading + "updated_at)"

    for a in range (0, len(data)): #Remove the headings from the dataset 
        sql_string = sql_string_heading + " VALUES ("
        for b in range (0, len(data[0])):
            if data[a][b] == "":
                sql_string = sql_string + 'NULL,'
            elif '"' in str(data[a][b]):
                sql_string = sql_string + '"' + str(data[a][b]).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string = sql_string + '"' + str(data[a][b]) + '",'
        sql_string = sql_string + "NOW())"
        mycursor.execute(sql_string)
    connection.commit()
    
# The function that collapses survey questions as a field rather than a label
# I have not added a question number to the survey questions (They are currently stored in the order of the excel spreadsheet but not indexed as such)

def import_database_survey(table_name, data, questions_headings, questions_data):
    # Get the headings from the mysql table
    sql_heading_Query = "SHOW COLUMNS FROM {}".format(table_name)
    mycursor.execute(sql_heading_Query)
    heading_data = mycursor.fetchall()
    
    sql_string_heading = "INSERT IGNORE INTO {} (".format(table_name) # INSERT IGNORE is used to avoid writing over previously inserted data points (This will ensure the timestamps are correct)
    for a in range (0, len(heading_data) - 1): # (-1) For the allowance of adding commas to the string
        sql_string_heading = sql_string_heading + heading_data[a][0] + ","
    sql_string_heading = sql_string_heading + "updated_at)"
    
    for a in range (0, len(data)):
        sql_string = sql_string_heading + " VALUES ("
        for b in range (0, len(data[0])):
            if data[a][b] == "":
                sql_string = sql_string + 'NULL,'
            elif '"' in str(data[a][b]):
                sql_string = sql_string + '"' + str(data[a][b]).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string = sql_string + '"' + str(data[a][b]) + '",'

        for c in range (0, len(questions_data[a])):
            sql_string_collapsed = sql_string

            temp_question_heading = questions_heading[c]
            if temp_question_heading == "":
                sql_string_collapsed = sql_string_collapsed + 'NULL,'
            elif '"' in str(temp_question_heading):
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_heading).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_heading) + '",'

            temp_question_answer = questions_data[a][c]
            if temp_question_answer == "":
                sql_string_collapsed = sql_string_collapsed + 'NULL,'
            elif '"' in str(temp_question_answer):
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_answer).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_answer) + '",'
            sql_string_collapsed = sql_string_collapsed + "NOW())"
            #print(sql_string_collapsed)
            mycursor.execute(sql_string_collapsed)
    connection.commit()

# CIRCADIEM Hospital

In [118]:
# Read the data file for import
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_LABELS_2023-08-16_1125.csv"

data = []
with open(r'{}'.format(filepath)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append(row)

## Create SQL Tables

In [91]:
# Core Details
mycursor.execute("""CREATE TABLE IF NOT EXISTS core_details (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Record_ID_Hospital_UMRN VARCHAR (8),
Hospital_recruiting_for_CIRCA_DIEM TEXT,
Inborn TEXT,
Sex TEXT, 
Date_and_time_of_birth DATETIME,
DOB DATE,
Expected_date_and_time_of_delivery DATETIME,
EDD DATE,
Gestational_age_w DECIMAL (5,2),
GA_group INTEGER,
GA_Strata TEXT, 
Is_infant_enrolled_in_any_other_interventional_RCT TEXT,
Check_other_RCT_choice_EVARO TEXT,
Check_other_RCT_choice_Protect TEXT,
Check_other_RCT_choice_ProPara TEXT,
Check_other_RCT_choice_PLUSS TEXT,
Check_other_RCT_choice_Cosi2 TEXT,
Check_other_RCT_choice_Provide TEXT,
Check_other_RCT_choice_WASH_T TEXT,
Check_other_RCT_choice_SurfsUp TEXT,
Check_other_RCT_choice_Other TEXT,
Check_other_RCT_choice_No_information TEXT,
Check_other_RCT_choice_Unknown TEXT,
Check_other_RCT_choice_Not_asked TEXT,
Check_other_RCT_choice_Asked_but_unknown TEXT,
Check_other_RCT_choice_Not_applicable TEXT,
Other_Trial TEXT,
Acronym_for_other_RCT TEXT,
Does_the_RCT_have_neurodevelopment_as_its_primary_outcome TEXT,
Does_infant_have_a_congenital_abnormality TEXT,
Please_provide_details_of_congenital_abnormality TEXT,
Is_infant_critically_ill_not_expected_to_survive TEXT,
Expected_discharge_hospital TEXT,
Can_infant_return_for_follow_up_studies_at_2_years TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Parental consent
mycursor.execute("""CREATE TABLE IF NOT EXISTS parental_consent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Informed_written_parental_consent TEXT,
Were_parentsguardian_approached_for_consent TEXT,
Why_researcher_unavailable TEXT,
Why_researcher_not_notified_of_admission TEXT,
Why_parents_unable_to_be_contacted_before_72_h TEXT,
Why_parents_dont_understand_English TEXT,
Why_infant_didnt_meet_eligibility_criteria TEXT,
Why_infant_enrolled_in_other_trial TEXT,
Why_Assigned_for_recruit_into_other_trial TEXT,
Why_infant_enrolled_in_maximum_number_of_studies_for_unit TEXT,
Why_other TEXT,
Why_No_information TEXT,
Why_Unknown TEXT,
Why_Not_asked TEXT,
Why_Asked_but_unknown TEXT,
Why_Not_applicable TEXT,
Other_reason_not_approached TEXT,
Type_of_Consent TEXT,
Date_and_time_of_enrol_ DATETIME,
Age_at_enrolment_h DECIMAL (5, 3),
Researcher_obtaining_main_consent TEXT,
Scanned_consent_form_filed TEXT,
Specific_name_of_consent_form_eg_CDXXXXX_mainconsent TEXT,
Consent_form_upload TEXT,
Has_parent_received_a_signed_copy_of_the_consent_form TEXT,
Date_parent_provided_copy_of_consent DATE,
Copy_of_the_consent_form_in_the_infants_medical_record TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Twin
mycursor.execute("""CREATE TABLE IF NOT EXISTS twin (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Is_this_infant_a_twin_or_triplet TEXT,
What_is_the_birth_order_of_this_infant TEXT,
Has_a_twin_or_triplet_already_been_randomised_to_the_study TEXT,
Study_ID_Auto_ID_of_previously_randomised_sibling_twintriplet TEXT,
Eligible_for_Randomisation TEXT,
Confirm_twin_sibling_eligible TEXT,
THIS_infants_randomised_study_group TEXT,
Please_select_the_allocated_group_for_THIS_TWIN TEXT,
Study_Number_from_SITE_PI_ TEXT,
Datetime_of_randomisation DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

# Parent Ethnicity
mycursor.execute("""CREATE TABLE IF NOT EXISTS parental_ethnicity (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Is_infant_indigenous TEXT,
Mothers_ethnicity TEXT,
Is_there_a_legal_second_parent_eg_father TEXT,
Second_parent_ethnicity TEXT,
Same_address_as_mother TEXT,
Second_parent_state TEXT,
Alternate_contact_connection_to_parents TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Maternal information
mycursor.execute("""CREATE TABLE IF NOT EXISTS maternal_information (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Date_of_last_menstrual_period DATE,
Date_of_earliest_antenatal_scan DATE,
Maternal_medical_problems_during_pregnancy TEXT,
Hypertension TEXT,
Asthma TEXT,
Diabetes TEXT,
Hyperthyroidism TEXT,
Hypothyroidism TEXT,
Epilepsy TEXT,
Other TEXT,
No_information TEXT,
Unknown TEXT,
Not_asked TEXT,
Asked_but_unknown TEXT,
Not_applicable TEXT,
Was_the_mother_on_thyroid_medication_during_pregnancy TEXT,
Clinical_chorioamnionitis_ TEXT,
maternal_intrapartum_fever_more_38_C TEXT,
uterine_tenderness TEXT,
maternal_tachycardia TEXT,
fetal_tachycardia TEXT,
foulpurulent_amniotic_fluid TEXT,
choice_not_checked TEXT,
choice_not_recorded TEXT,
choice_No_information TEXT,
choice_Unknown TEXT,
choice_Not_asked TEXT,
choice_Asked_but_unknown TEXT,
choice_Not_applicable TEXT,
Histological_chorioamnionitis TEXT,
Placental_histo_report_filename TEXT,
Placental_histology_report TEXT,
Date_and_time_of_membrane_rupture DATETIME,
ROM_Duration DECIMAL (5, 1),
Antenatal_Steroids TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Birth information
mycursor.execute("""CREATE TABLE IF NOT EXISTS birth_information (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Head_circumference_cm DECIMAL (5, 1),
Birth_weight_g INTEGER,
Birth_weight_Z_score TEXT,
Birthwt_centile TEXT,
IUGR INT,
Delivery_mode TEXT,
APGAR_at_1_min TEXT,
APGAR_at_5_min TEXT,
APGAR_at_10_min TEXT,
Cord_blood_gas TEXT,
Cord_gas_source_use_cord_arterial_sample_if_available TEXT,
Cord_Blood_Gas__pH DECIMAL (10, 3),
Cord_blood_gas__pCO2_mmHg DECIMAL (10, 3),
Cord_blood_gas__BE DECIMAL (10, 3),
Cord_blood_gas__Lactate_mmolL DECIMAL (10, 3),
Meconium_stained_amniotic_fluid TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Pregnancy information
mycursor.execute("""CREATE TABLE IF NOT EXISTS pregnancy_information (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Date_pregnancy_survey_started DATE,
Have_you_already_completed_this_survey_for_sibling TEXT,
During_the_last_month_of_pregnancy_bed_time TIME,
During_the_last_month_of_pregnancy_wake_time TIME,
Maximum_sleep_time DECIMAL (5, 1),
How_would_you_rate_the_quality_of_your_sleep TEXT,
Did_your_occupation_require_you_to_do_any_shift_work TEXT,
Did_your_shift_work_involve_overnight_shifts TEXT,
Typical_start_time_for_overnight_shifts TEXT,
Typical_time_overnight_shifts_ended TEXT,
Typical_number_of_night_shifts_per_week TEXT,
Did_you_stop_overnight_shift_work_during_pregnancy TEXT,
About_when_did_you_stop_shift_work_in_pregnancy TEXT,
Comments_about_your_shiftwork_during_pregnancy TEXT,
Have_you_taken_nicotine_substance TEXT,
Did_you_take_other_nicotine_substance_during_pregnancy TEXT,
During_your_pregnancy_how_many_cigarettes_per_day TEXT,
Did_you_stop_smoking_before_the_end_of_your_pregnancy TEXT,
How_many_weeks_pregnant_were_you_when_you_stopped_smoking TEXT,
Mother_basic_health_and_lifestyle_questionnaire_upload_ TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Growth information
mycursor.execute("""CREATE TABLE IF NOT EXISTS growth (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Measurement_date_and_time TEXT,
Corrected_age_at_growth_measurement TEXT,
Crownheel_length TEXT,
Length_centile TEXT,
Length_Z_score TEXT,
Head_circumference TEXT,
HC_centile TEXT,
HC_Z_score TEXT,
Infants_weight INTEGER,
Weight_centile TEXT,
Weight_Z_score TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Discharge Information
mycursor.execute("""CREATE TABLE IF NOT EXISTS discharge_information (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Did_infant_have_a_Pea_Pod_assessment_prior_to_discharge TEXT,
No_PeaPod_at_recruitment_hospital TEXT,
Infant_discharged_before_pea_pod_assessment_could_be_arranged TEXT,
no_substudy_consent TEXT,
other_reason TEXT,
No_information TEXT,
Unknown TEXT,
Not_asked TEXT,
Asked_but_unknown TEXT,
Not_applicable TEXT,
PeaPod_test_date_and_time DATETIME,
Peapod_researcher TEXT,
Fat_percent DECIMAL (10, 5),
Fat_free_mass_percent DECIMAL (10, 5),
Fat_mass_in_kg DECIMAL (10, 5),
Fat_free_mass_in_kg DECIMAL (10, 5),
Body_mass_in_kg DECIMAL (10, 5),
Body_volume_in_L DECIMAL (10, 5),
Body_density_ DECIMAL (10, 5),
Fat_mass_density_in_kgL DECIMAL (10, 5),
Fat_free_mass_density_in_kgL DECIMAL (10, 5),
Body_surface_area_in_cm2 DECIMAL (10, 5),
Thoracic_gas_volume_in_L DECIMAL (10, 5),
Peapod_report_filename TEXT,
Peapod_report_upload TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Light Audit
mycursor.execute("""CREATE TABLE IF NOT EXISTS light_audit (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Repeat_Instrument TEXT,
_Was_light_audit_obtained TEXT,
Reason_no_light_monitoring_file_obtained TEXT,
light_audit_date DATE,
Light_log_filename TEXT,
Light_data_log_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Sound Audit
mycursor.execute("""CREATE TABLE IF NOT EXISTS sound_audit (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Repeat_Instrument TEXT,
_Was_sound_audit_obtained TEXT,
Reason_no_sound_file_obtained TEXT,
Sound_audit_date DATE,
Sound_log_filename TEXT,
Sound_log_file_upload TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# 36w_information
mycursor.execute("""CREATE TABLE IF NOT EXISTS 36w_information (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Date_36_w_PMA_respiratory_records_obtained TEXT,
Current_respiratory_support_at_36_w_PMA_choice_None TEXT,
Current_respiratory_support_at_36_w_PMA_choice_PBF TEXT,
Current_respiratory_support_at_36_w_PMA_choice_HHF TEXT,
Current_respiratory_support_at_36_w_PMA_choice_CPAP TEXT,
Current_respiratory_support_at_36_w_PMA_choice_MV TEXT,
Current_respiratory_support_at_36_w_PMA_choice_No_information TEXT,
Current_respiratory_support_at_36_w_PMA_choice_Unknown TEXT,
Current_respiratory_support_at_36_w_PMA_choice_Not_asked TEXT,
Current_respiratory_support_at_36_w_PMA_choice_Asked_but_unknown TEXT,
Current_respiratory_support_at_36_w_PMA_choice_Not_applicable TEXT,
O2_flow_in_mLmin TEXT,
HHF_Flow TEXT,
CPAP_Pressure TEXT,
Most_recent_Hb TEXT,
Modified_walsh_oxygen_reduction_air_trial TEXT,
Avge_FiO2__from_shift_test_ TEXT,
Avge_SpO2_from_shift_test INTEGER,
Altitude1 INTEGER,
Altitude2 INTEGER,
File_name_for_24_h_observations TEXT,
24_h_observations TEXT,
Average_SpO2_over_24_h TEXT,
Average_FiO2_over_24_h INTEGER,
Predicted_24_hour_Shift_average_paired_SpO2PiO2 TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Substudy
mycursor.execute("""CREATE TABLE IF NOT EXISTS substudy (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Biological_substudy_consent TEXT,
Researcher_consenting_biol_substudy TEXT,
Date_of_biol_substudy_consent DATE,
Biol_substudy_consent_filename TEXT,
Biological_substudy_consent_file TEXT,
Physiological_substudy_consent TEXT,
Researcher_consenting_physiology_substudy TEXT,
Date_of_physiology_substudy_consent TEXT,
Physiology_substudy_consent_filename TEXT,
Physiological_substudy_consent_file TEXT,
MRI_substudy_consent TEXT,
Researcher_consenting_MRI_substudy TEXT,
Date_of_MRI_substudy_consent TEXT,
MRI_substudy_consent_filename TEXT,
MRI_substudy_consent_file TEXT,
Sleep_substudy_consent TEXT,
Researcher_consenting_sleep_substudy TEXT,
Date_of_sleep_substudy_consent TEXT,
Sleep_substudy_consent_filename TEXT,
Sleep_substudy_consent_file TEXT,
Social_behaviour_substudy_consent TEXT,
Researcher_consenting_behaviour_substudy TEXT,
Date_of_behaviour_substudy_consent TEXT,
Social_behaviour_substudy_consent_filename TEXT,
Behaviour_substudy_consent_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Respiratry Information
mycursor.execute("""CREATE TABLE IF NOT EXISTS respiratory_information (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Main_initial_respiratory_diagnosis TEXT,
Exogenous_surfactant TEXT,
No_of_surfactant_doses TEXT,
Airleak_requiring_drainage TEXT,
Nitric_Oxide_NO_used TEXT,
NO_duration TEXT,
Extracorporeal_Membrane_Oxygenation_ECMO TEXT,
Dur_HHF INTEGER,
Dur_CPAP INTEGER,
Dur_NIV INTEGER,
Dur_IPPV_h INTEGER,
Dur_HFV INTEGER,
Date_last_resp_support DATETIME,
Respiratory_support_at_36w_PMA TEXT,
Systemic_postnatal_steroids_ TEXT,
Hours_of_supplemental_O2 INTEGER,
PDA TEXT,
Type_of_treatment_for_PDA_choice_None_or_fluid_restrict TEXT,
Type_of_treatment_for_PDA_choice_pharmacological TEXT,
Type_of_treatment_for_PDA_choice_surgery TEXT,
Type_of_treatment_for_PDA_choice_No_information TEXT,
Type_of_treatment_for_PDA_choice_Unknown TEXT,
Type_of_treatment_for_PDA_choice_Not_asked TEXT,
Type_of_treatment_for_PDA_choice_Asked_but_unknown TEXT,
Type_of_treatment_for_PDA_choice_Not_applicable TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Head Ultrasound Information
mycursor.execute("""CREATE TABLE IF NOT EXISTS head_ultrasound (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Head_ultrasound_done TEXT,
Date_of_Head_Ultrasound DATETIME,
Maximum_grade_of_left_sided_periventricular_haemorrhage TEXT,
Maximum_grade_of_right_sided_periventricular_haemorrhage TEXT,
Cerebral_cysts TEXT,
Cerebral_cysts_Left TEXT,
Cerebral_cysts_Right TEXT,
File_name_for_USS_report TEXT,
Ultrasound_report TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Near term MRI
mycursor.execute("""CREATE TABLE IF NOT EXISTS nearterm_MRI (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Was_near_term_MRI_done TEXT,
Date_and_time_of_pre_dischargenear_term_MRI DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

# NEC
mycursor.execute("""CREATE TABLE IF NOT EXISTS nec (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Was_NEC_diagnosed TEXT,
Datetime_of_NEC_diagnosis TEXT,
NEC_treatment_choice_none TEXT,
NEC_treatment_choice_medical TEXT,
NEC_treatment_choice_surgery TEXT,
NEC_treatment_choice_No_information TEXT,
NEC_treatment_choice_Unknown TEXT,
NEC_treatment_choice_Not_asked TEXT,
NEC_treatment_choice_Asked_but_unknown TEXT,
NEC_treatment_choice_Not_applicable TEXT,
NEC_stage TEXT,
TPN_initiated TEXT,
TPN_hours INTEGER,
TPN_days INTEGER,
Datetime_TPN_ceased DATETIME,
Date_achieved_enteral_feed_120_mLkg TEXT,
Breast_milk_feeding_at_discharge TEXT,
Worst_ROP TEXT,
ROP_Plus TEXT,
Surgical_ROP_Rx TEXT,
VEGF TEXT,
Culture_ve_sepsis TEXT,
No_of_culture_ve_sepsis_episodes TEXT,
Phototherapy_for_Jaundice TEXT,
Maximum_SBR INTEGER,
Max_SBR_date_and_time TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Hospital Information
mycursor.execute("""CREATE TABLE IF NOT EXISTS hospital_information (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Datetime_of_discharge_from_primary_recruiting_hospital DATETIME,
Where_was_infant_discharge_to_from_the_primary_recruiting_site TEXT,
Secondary_hospital TEXT,
Other_hospital_Secondary TEXT,
Where_was_infant_discharged_to_after_2nd_hospital TEXT,
Date_and_time_infant_discharged_from_second_hospital DATETIME,
Third_hospital TEXT,
Other_hospital_Third TEXT,
Where_was_infant_discharged_to_after_3rd_hospital TEXT,
Date_and_time_infant_discharged_from_third_hospital DATETIME,
Fourth_hospital TEXT,
Other_hospital_Fourth TEXT,
Where_was_infant_discharged_to_after_4th_hospital TEXT,
Date_and_time_infant_discharged_from_fourth_hospital DATETIME,
Fifth_hospital TEXT,
Other_hospital_Fifth TEXT,
Date_and_time_infant_discharged_from_fifth_hospital DATETIME,
Date_and_time_infant_discharged_home DATETIME,
Duration_of_hospitalisation_Days DECIMAL (5, 1),
Discharge_comment TEXT,
O2_ceased_in_hospital TEXT,
Datetime_O2_ceased TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Survival
mycursor.execute("""CREATE TABLE IF NOT EXISTS survival (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Has_infant_died TEXT,
Date_and_time_of_death_ DATETIME,
Primary_cause_of_death TEXT,
postnatal_age_at_time_of_death TEXT,
PMA_at_time_of_death TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Adverse Events
mycursor.execute("""CREATE TABLE IF NOT EXISTS adverse_events (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Was_there_an_adverse_event_related_to_eye_mask_or_ear_plugs TEXT,
DateTime_of_AE_noted TEXT,
Study_device_involved_in_AE_choice_eye_mask TEXT,
Study_device_involved_in_AE_choice_ear_plugs TEXT,
Study_device_involved_in_AE_choice_No_information TEXT,
Study_device_involved_in_AE_choice_Unknown TEXT,
Study_device_involved_in_AE_choice_Not_asked TEXT,
Study_device_involved_in_AE_choice_Asked_but_unknown TEXT,
Study_device_involved_in_AE_choice_Not_applicable TEXT,
AE_Type_choice_skin_damage_from_use_of_eye_mask_or_ear_plugs TEXT,
AE_Type_choice_infection_associated_with_eye_mask_or_ear_plugs TEXT,
AE_Type_choice_safety_issue_with_eye_mask_or_ear_plugs TEXT,
AE_Type_choice_No_information TEXT,
AE_Type_choice_Unknown TEXT,
AE_Type_choice_Not_asked TEXT,
AE_Type_choice_Asked_but_unknown TEXT,
AE_Type_choice_Not_applicable TEXT,
Was_AE_life_threatening TEXT,
Describe_skin_damage TEXT,
Describe_infection TEXT,
Describe_safety_issue TEXT,
Was_there_any_other_adverse_event TEXT,
Describe_other_adverse_event TEXT,
AE_MedDRA_coding_term_1 TEXT,
AE_MedDRA_coding_term_2 TEXT,
AE_MedDRA_coding_term_3 TEXT,
Was_other_adverse_event_life_threatening TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Protocol Deviation
mycursor.execute("""CREATE TABLE IF NOT EXISTS protocol_deviation (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Was_there_a_protocol_deviation_recorded_for_this_participant TEXT,
Date_of_deviation DATE,
Date_identified DATE,
Deviation_description TEXT,
Deviation_Code__ TEXT,
Deviation_Code_Other TEXT,
Resulted_in_Adverse_Event TEXT,
Did_participant_continue_in_study TEXT,
Date_of_withdrawal DATE,
Ethics_Reporting_required TEXT,
Ethics_report_date TEXT,
PI_signature TEXT,
Protocol_deviation_form_upload TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Withdrawal
mycursor.execute("""CREATE TABLE IF NOT EXISTS withdrawal (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Study_withdrawal TEXT,
Withdrawal_datetime DATETIME,
Withdrawal_form TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Hospital Duration
mycursor.execute("""CREATE TABLE IF NOT EXISTS hospital_duration (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Hospital_duration_primary_days_Duration DECIMAL (5,1),
Secondary_hospital_duration_days DECIMAL (5,1),
Interim_secondary_hospital_site__  TEXT,
Third_hospital_duration_days DECIMAL (5,1),
Interim_third_hospital_site__  TEXT,
Fourth_hospital_duration_days DECIMAL (5,1),
Interim_fourth_hospital_site TEXT,
Hospital_duration_primary_days_Duration_until_discharged_home DECIMAL (5,1),
Discharge_instruments_complete INTEGER,
Survival INTEGER,
Withdraw INTEGER,
Complete TEXT,
updated_at TIMESTAMP)""")

## Import Data

In [9]:
# Initial 6 columns refer to linking fields
# ï»¿"Auto_ID"
# Event_Name
# Repeat_Instrument
# Repeat_Instance
# Data_Access_Group
# Survey_Identifier

### Core details

In [10]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "D0-3":
        temp_data = data[a][0:2] + data[a][6:41]
        import_data.append(temp_data)
        
import_database("core_details", import_data)

### Parental consent

In [11]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "D0-3":
        temp_data = data[a][0:2] + data[a][68:96]
        import_data.append(temp_data)
        
import_database("parental_consent", import_data)

### Twins

In [12]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "D0-3":
        temp_data = data[a][0:2] + data[a][96:107]
        import_data.append(temp_data)
        
import_database("twin", import_data)

### Parent ethnicity

In [13]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "D0-3":
        temp_data = data[a][0:2] + data[a][107:115]
        import_data.append(temp_data)
        
import_database("parental_ethnicity", import_data)

### Maternal Information

In [14]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "D0-3":
        temp_data = data[a][0:2] + data[a][115:151]
        import_data.append(temp_data)
        
import_database("maternal_information", import_data)

### Birth Information

In [15]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "D0-3":
        temp_data = data[a][0:2] + data[a][151:168]
        import_data.append(temp_data)
        
import_database("birth_information", import_data)

### Pregnancy Information

In [120]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "D0-3":
        temp_data = data[a][0:2] + data[a][168:190]
        import_data.append(temp_data)
        
import_database("pregnancy_information", import_data)

### Growth

In [16]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    temp_data = data[a][0:2] + data[a][237:250]
    temp_string = ''
    for a in range (2, len(temp_data)):
        temp_string = temp_string + temp_data[a]
        
    if temp_string != 'Incomplete':
        import_data.append(temp_data)
        
import_database("growth", import_data)

### Discharge

In [17]:
# Check to see if there is any more information in different event_name

# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "36w":
        temp_data = data[a][0:2] + data[a][250:276]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("discharge_information", import_data)

### Light Audit

In [18]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    repeat_instrument = data[a][2]
    
    if repeat_instrument == "Light Audit":
        temp_data = data[a][0:3] + data[a][276:282]
        import_data.append(temp_data)
        
import_database("light_audit", import_data)

### Sound Audit

In [19]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    repeat_instrument = data[a][2]
    
    if repeat_instrument == "Sound Audit":
        temp_data = data[a][0:3] + data[a][282:288]
        import_data.append(temp_data)
        
import_database("sound_audit", import_data)

### 36w Information

In [20]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "36w":
        temp_data = data[a][0:2] + data[a][288:314]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("36w_information", import_data)

### Substudy

In [21]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Disch":
        temp_data = data[a][0:2] + data[a][314:340]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("substudy", import_data)

### Respiratory Information

In [41]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Disch":
        temp_data = data[a][0:2] + data[a][340:367]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("respiratory_information", import_data)

### Head Ultrasound

In [42]:
# This has multiple occurances in different events

# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    head_ultrasound = data[a][367]
    
    if head_ultrasound != None:
        temp_data = data[a][0:2] + data[a][367:377]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("head_ultrasound", import_data)

### Near term MRI

In [47]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Disch":
        temp_data = data[a][0:2] + data[a][377:380]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("nearterm_MRI", import_data)

### NEC

In [52]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Disch":
        temp_data = data[a][0:2] + data[a][380:408]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("nec", import_data)

### Hospital information

In [56]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Disch":
        temp_data = data[a][0:2] + data[a][408:432]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("hospital_information", import_data)

### Survival

In [59]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Survival":
        temp_data = data[a][0:2] + data[a][432:438]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("survival", import_data)

### Adverse event

In [80]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Adverse Event":
        temp_data = data[a][0:2] + data[a][438:466]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("adverse_events", import_data)

### Protocol deviation

In [ ]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Protocol Deviation":
        temp_data = data[a][0:2] + data[a][466:480]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("adverse_events", import_data)

### Withdrawal

In [84]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Study Withdrawal":
        temp_data = data[a][0:2] + data[a][480:484]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("withdrawal", import_data)

### Hospital Duration

In [92]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    hosp_duration = data[a][484]
    
    if hosp_duration != None:
        temp_data = data[a][0:2] + data[a][484:496]
        
        temp_string = ''
        for a in range (2, len(temp_data)):
            temp_string = temp_string + temp_data[a]
            
        if temp_string != '':
            import_data.append(temp_data)
        
import_database("hospital_duration", import_data)

# CIRCADIEM FollowUp

In [9]:
# Read the data file for import
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_LABELS_2023-09-27_0929.csv"

data = []
with open(r'{}'.format(filepath)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append(row)

## Create SQL Tables

In [7]:
# Core Details
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_core (
Study_ID TEXT,
Event_Name TEXT,
Repeat_Instrument TEXT,
Repeat_Instance TEXT,
Data_Access_Group TEXT,
Survey_Identifier TEXT,
Record_ID_Hospital_UMRN TEXT,
Hospital_recruiting_for_CIRCA_DIEM TEXT,
Sex TEXT,
DOB DATE,
EDD DATE,
GA_Strata TEXT,
Check_other_RCT_choice_EVARO TEXT,
Check_other_RCT_choice_Protect TEXT,
Check_other_RCT_choice_ProPara TEXT,
Check_other_RCT_choice_PLUSS TEXT,
Check_other_RCT_choice_Cosi2 TEXT,
Check_other_RCT_choice_Provide TEXT,
Check_other_RCT_choice_WASHT TEXT,
Check_other_RCT_choice_SurfsUp TEXT,
Check_other_RCT_choice_Other TEXT,
Check_other_RCT_choice_No_information TEXT,
Check_other_RCT_choice_Unknown TEXT,
Check_other_RCT_choice_Not_asked TEXT,
Check_other_RCT_choice_Asked_but_unknown TEXT,
Check_other_RCT_choice_Not_applicable TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_contact_info
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_contact_info (
Study_ID TEXT,
Event_Name TEXT,
Infant_First_Name TEXT,
Infants_surname_in_hospital TEXT,
Infant_Last_Name TEXT,
Is_infant_indigenous TEXT,
Mothers_medical_record_number TEXT,
Mothers_first_Name TEXT,
Mothers_Surname TEXT,
Mothers_ethnicity TEXT,
UnitNumber_and_Street TEXT,
Suburb TEXT,
State TEXT,
Post_Code TEXT,
Mothers_Home_Phone_Number TEXT,
Mothers_Mobile_PhNo TEXT,
Mothers_Email TEXT,
Second_Parent_Preferred_First_Name TEXT,
Second_Parent_Last_Name TEXT,
Second_Parent_Ethnicity TEXT,
Second_Parent_Address TEXT,
Second_Parent___FlatNumber_Street_Address TEXT,
Second_Parent_Suburb TEXT,
Second_Parent_State TEXT,
Second_Parent_Post_Code TEXT,
Second_Parent_Home_Phone_Number TEXT,
Second_Parent_Mobile_PhNo TEXT,
Second_Parent_Email TEXT,
Alternate_contact_preferred_first_name TEXT,
Alternate_contact_last_name TEXT,
Alternate_contact_connection_to_parents TEXT,
Alternate_contact_Mobile_PhNo TEXT,
Infants_paediatrician_First_Name TEXT,
Infants_paediatrician_Surname TEXT,
First_name_of_Infants_GP TEXT,
Surname_of_infants_GP TEXT,
GPs_clinic_name TEXT,
Number_and_street_of_GP_practice TEXT,
GP_Suburb TEXT,
GP_Postcode TEXT,
GP_Contact_Number TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_measurements
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_measurements (
Study_ID TEXT,
Event_Name TEXT,
Measurement_date_and_time DATETIME,
Corrected_age_at_growth_measurement INTEGER,
Crownheel_length DECIMAL (5,1),
Length_centile  DECIMAL (5,1),
Length_Z_score DECIMAL (5,1),
Head_circumference DECIMAL (5,1),
HC_centile DECIMAL (5,1),
HC_Z_score DECIMAL (5,1),
Infants_weight INTEGER,
Weight_centile DECIMAL (5,1),
Weight_Z_score DECIMAL (5,1),
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_pregnancy_information
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_pregnancy_information (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Date_pregnancy_survey_started DATE,
Have_you_already_completed_this_survey_for_sibling TEXT,
During_the_last_month_of_pregnancy_bed_time TIME,
During_the_last_month_of_pregnancy_wake_time TIME,
Maximum_sleep_time DECIMAL (5, 1),
How_would_you_rate_the_quality_of_your_sleep TEXT,
Did_your_occupation_require_you_to_do_any_shift_work TEXT,
Did_your_shift_work_involve_overnight_shifts TEXT,
Typical_start_time_for_overnight_shifts TEXT,
Typical_time_overnight_shifts_ended TEXT,
Typical_number_of_night_shifts_per_week TEXT,
Did_you_stop_overnight_shift_work_during_pregnancy TEXT,
About_when_did_you_stop_shift_work_in_pregnancy TEXT,
Comments_about_your_shiftwork_during_pregnancy TEXT,
Have_you_taken_nicotine_substance TEXT,
Did_you_take_other_nicotine_substance_during_pregnancy TEXT,
During_your_pregnancy_how_many_cigarettes_per_day TEXT,
Did_you_stop_smoking_before_the_end_of_your_pregnancy TEXT,
How_many_weeks_pregnant_were_you_when_you_stopped_smoking TEXT,
Mother_basic_health_and_lifestyle_questionnaire_upload_ TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_substudy
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_substudy (
Study_ID TEXT,
Event_Name TEXT,
Infant_Sleep_Substudy TEXT,
Sleep_substudy_consent_filename TEXT,
MRI_substudy_consent TEXT,
Name_of_MRI_Substudy_Consent_Form__assumed_pdf TEXT,
MRI_substudy_consent_file TEXT,
Autism_substudy_consent TEXT,
Name_of_Autism_Substudy_Consent_Form__assumed_pdf TEXT,
Autism_substudy_consent_form TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_survival
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_survival (
Study_ID TEXT,
Event_Name TEXT,
Has_infant_died TEXT,
Date_of_death_ DATE,
Primary_cause_of_death TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_2mSurvey
mycursor.execute("""CREATE TABLE IF NOT EXISTS 2mSurvey (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Date_of_survey DATETIME,
Complete TEXT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

## Import Data

### Core

In [101]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Term":
        temp_data = data[a][0:2] + data[a][2:27]
        import_data.append(temp_data)
        
import_database("fu_core", import_data)

### Contact Info

In [110]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Term":
        temp_data = data[a][0:2] + data[a][27:67]
        import_data.append(temp_data)
        
import_database("fu_contact_info", import_data)

### Measurements

In [115]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Term":
        temp_data = data[a][0:2] + data[a][67:79]
        import_data.append(temp_data)
        
import_database("fu_measurements", import_data)

### Pregnancy Information

In [129]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Term":
        temp_data = data[a][0:2] + data[a][79:101]
        import_data.append(temp_data)
        
import_database("fu_pregnancy_information", import_data)

### Substudy

In [133]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Term":
        temp_data = data[a][0:2] + data[a][101:110]
        import_data.append(temp_data)
        
import_database("fu_substudy", import_data)

### Survival

In [135]:
# Retrieve the appropriate columns and rows
import_data = []
for a in range (0, len(data)):
    event_name = data[a][1]
    
    if event_name == "Term":
        temp_data = data[a][0:2] + data[a][110:114]
        import_data.append(temp_data)
        
import_database("fu_survival", import_data)

### 2m_survey

In [12]:
# Collapse the Questions down to a field.
# Retrieve the appropriate columns and rows
import_data = []
questions_data = []

for a in range (0, len(data)):
    event_name = data[a][1]
    repeat_instrument = data[a][2]
    
    if event_name == "cPNA2m" and repeat_instrument == "":
        # Survey date / survey timestamp / complete
        temp_data = data[a][0:2] + data[a][114:116] + data[a][177:178]
        questions_data_row = data[a][116:177]
        import_data.append(temp_data)
        questions_data.append(questions_data_row)
        
# Gather the headings of questions of the survey
questions_headings = data[0][116:177]

# Impot survey structured data into the database
import_database_survey("2mSurvey", import_data, questions_headings, questions_data)

In [23]:
# View all the headings of the the fields separated by the "Complete" field for different forms
for a in range (178, len(data[0])):
    temp = data[0][a].replace(" ", "_").replace("(","").replace(")","").replace("?","").replace("=", "_").replace("+", "").replace(".", "").replace("/", "")
    print(a, temp)
    
    if temp == "Complete":
        print("--------------------------------------------------------------------------------")

178 Survey_Timestamp
179 Date_of_survey
180 Does_your_baby_make_high-pitched_squeals
181 When_playing_with_sounds,_does_your_baby_make_grunting,_growling,_or_other_deep-toned_sounds
182 If_you_call_your_baby_when_you_are_out_of_sight,_does_she_look_in_the_direction_of_your_voice
183 When_a_loud_noise_occurs,_does_your_baby_turn_to_see_where_the_sound_came_from
184 Does_your_baby_make_sounds_like_'da,'_'ga,'_'ka,'_and_'ba'
185 If_you_copy_the_sounds_your_baby_makes,_does_your_baby_repeat_the_same_sounds_back_to_you
186 While_your_baby_is_on_his_back,_does_your_baby_lift_his_legs_high_enough_to_see_his_feet
187 When_your_baby_is_on_her_tummy,_does_she_straighten_both_arms_and_push_her_whole_chest_off_the_bed_or_floor
188 Does_your_baby_roll_from_his_back_to_his_tummy,_getting_both_arms_out_from_under_him
189 When_you_put_your_baby_on_the_floor,_does_she_lean_on_her_hands_while_sitting_If_she_already_sits_up_straight_without_leaning_on_her_hands,_mark_'yes'_for_this_item
190 If_you_hold_b

In [30]:
# Release Memory
%reset -f